In [1]:
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np
from scipy.interpolate import interp1d
import matplotlib.pyplot as plt

# === your GS reference data ===
gs_data = {
    "start": [11703,14692,23020,23340,27780,28900,30840,32500,
              33740,35480,38220,40160,41460,43340,46860,49280,
              54220,55000,55800,58040,58280,59080,59440,64100,
              69620,72340,76440,84760,85060,90040,
              104040,104520,106750,108280,115370],
    "end":   [12896,22900,23220,27540,28600,30600,32040,33360,
              34740,36580,39900,40800,42240,44280,48340,49600,
              54900,55400,56500,58160,58560,59300,63840,69400,
              70380,74100,77760,84960,87600,90140,
              104380,105440,106900,110640,119140]
}

# === load & preprocess ===
file_path = r"D:\VScode\bipolar_seesaw_CCM\other_data\monsoon.xlsx"
df = pd.read_excel(file_path)
df['age'] = df['age'] * 1000

# interpolate to 10-yr grid
new_age = np.arange(0, int(df['age'].max())+10, 10)
f = interp1d(df['age'], df['d18O'], kind='nearest',
             bounds_error=False, fill_value=1)
interpolated = f(new_age)
new_df = pd.DataFrame({'age': new_age, 'd18O': interpolated})

# low-frequency smooth
new_df['smoothed'] = new_df['d18O'].rolling(1000, center=True, min_periods=1).mean()
# millennial signal
new_df['d18O_ms'] = new_df['d18O'] - new_df['smoothed']
# extra smooth + first difference
new_df['d18O_ms']   = new_df['d18O_ms'].rolling(50, center=True, min_periods=1).mean()
new_df['d18O_diff'] = new_df['d18O_ms'].diff().fillna(0).rolling(15, center=True, min_periods=1).mean()

# === build figure with 2 rows ===
fig = make_subplots(
    rows=2, cols=1,
    shared_xaxes=True,
    vertical_spacing=0.05,
    subplot_titles=("d18O vs Age", "d18O_diff vs Age")
)

# top panel: d18O
fig.add_trace(
    go.Scatter(x=new_df['age'], y=new_df['d18O_ms'], mode='lines', name='d18O'),
    row=1, col=1
)

# bottom panel: diff
fig.add_trace(
    go.Scatter(x=new_df['age'], y=new_df['d18O_diff'],
               mode='lines', name='d18O_diff', line=dict(color='red')),
    row=2, col=1
)

# === add GS vertical strips to the diff panel ===
for s, e in zip(gs_data['start'], gs_data['end']):
    fig.add_vrect(
        x0=s, x1=e,
        fillcolor='LightSalmon', opacity=0.3,
        line_width=0,
        row=2, col=1
    )

# === layout tweaks ===
fig.update_layout(
    height=900, width=1800,
    showlegend=False,
    title_text="Monsoon δ¹⁸O and Its First Difference (with GS periods)"
)

# axes labels
fig.update_xaxes(title_text="Age (years)", row=2, col=1)
fig.update_yaxes(title_text="d18O",      row=1, col=1)
fig.update_yaxes(title_text="d18O_diff", row=2, col=1)

# add range-slider on the diff panel
fig.update_xaxes(rangeslider=dict(visible=True), row=2, col=1)

fig.show()


FileNotFoundError: [Errno 2] No such file or directory: 'D:\\VScode\\bipolar_seesaw_CCM\\other_data\\monsoon.xlsx'

 find peaks corresponse to start, valley corresponds to end, set the tolerrence to be 50 years

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import find_peaks


# 1. reference GS data (yrs)
gs_data = {
    "start": [11703,14692,23020,23340,27780,28900,30840,32500,
              33740,35480,38220,40160,41460,43340,46860,49280,
              54220,55000,55800,58040,58280,59080,59440,64100,
              69620,72340,76440,84760,85060,90040,
              104040,104520,106750,108280,115370],
    "end":   [12896,22900,23220,27540,28600,30600,32040,33360,
              34740,36580,39900,40800,42240,44280,48340,49600,
              54900,55400,56500,58160,58560,59300,63840,69400,
              70380,74100,77760,84960,87600,90140,
              104380,105440,106900,110640,119140]
}
gs_intervals = np.column_stack((gs_data['start'], gs_data['end']))

# 2. enforce 1:1 alternation
def pair_peaks_valleys(peaks, valleys):
    peaks, valleys = np.sort(peaks), np.sort(valleys)
    i, j = 0, 0
    pairs = []
    # skip any valley before first peak
    while j < len(valleys) and valleys[j] < peaks[0]:
        j += 1
    while i < len(peaks) and j < len(valleys):
        s = peaks[i]
        # advance valley until > s
        while j < len(valleys) and valleys[j] < s:
            j += 1
        if j == len(valleys): break
        e = valleys[j]
        pairs.append((s, e))
        # skip peaks up to this valley
        i += 1
        while i < len(peaks) and peaks[i] <= e:
            i += 1
        j += 1
    return np.array(pairs)  # shape (N,2)

# 3. new F1 based on overlap
def compute_f1_from_intervals(ages, known_intervals, pred_intervals):
    # build binary masks
    known_mask = np.zeros_like(ages, dtype=bool)
    for s,e in known_intervals:
        known_mask[(ages >= s) & (ages <= e)] = True
    pred_mask = np.zeros_like(ages, dtype=bool)
    for s,e in pred_intervals:
        pred_mask[(ages >= s) & (ages <= e)] = True

    tp = np.sum(pred_mask & known_mask)
    fp = np.sum(pred_mask & ~known_mask)
    fn = np.sum(~pred_mask & known_mask)

    prec = tp/(tp+fp) if tp+fp>0 else 0
    rec  = tp/(tp+fn) if tp+fn>0 else 0
    return 2*prec*rec/(prec+rec) if (prec+rec)>0 else 0

# 4. crop ≤120 ka
df_crop   = new_df[new_df['age'] <= 120_000].copy()
ages_crop = df_crop['age'].values
diff_crop = df_crop['d18O_diff'].values

# 5. sweep thresholds to maximize new F1
upper_grid = np.linspace(np.quantile(diff_crop,0.50),
                         np.quantile(diff_crop,0.99), 100)
lower_grid = np.linspace(np.quantile(diff_crop,0.01),
                         np.quantile(diff_crop,0.50), 100)

best_f1 = -1
for ut in upper_grid:
    # candidate starts
    pk_idx, _ = find_peaks(diff_crop, height=ut, distance=50)
    pk_ages   = ages_crop[pk_idx]
    for lt in lower_grid:
        # candidate ends
        vl_idx, _ = find_peaks(-diff_crop, height=-lt, distance=50)
        vl_ages   = ages_crop[vl_idx]
        # enforce alternation
        pairs = pair_peaks_valleys(pk_ages, vl_ages)
        if pairs.size == 0:
            continue
        f1 = compute_f1_from_intervals(ages_crop, gs_intervals, pairs)
        if f1 > best_f1:
            best_f1 = f1
            best = {
                'upper': ut, 'lower': lt,
                'pairs_crop': pairs,
                'f1': f1
            }

print(f"Best thresholds → upper={best['upper']:.3e}, lower={best['lower']:.3e}, F1={best['f1']:.3f}")

# 6. apply to full record
ages_full = new_df['age'].values
diff_full = new_df['d18O_diff'].values
pkf, _ = find_peaks(diff_full,  height=best['upper'], distance=50)
vlf, _ = find_peaks(-diff_full, height=-best['lower'], distance=50)
pairs_full = pair_peaks_valleys(ages_full[pkf], ages_full[vlf])

# build square waves
sq_crop = np.ones_like(ages_crop)
for s,e in best['pairs_crop']:
    sq_crop[(ages_crop>=s)&(ages_crop<=e)] = -1

sq_full = np.ones_like(ages_full)
for s,e in pairs_full:
    sq_full[(ages_full>=s)&(ages_full<=e)] = -1

# 7a. plot ≤120 ka
fig, ax = plt.subplots(figsize=(20,4))
for s,e in gs_intervals:      ax.axvspan(s, e, alpha=0.2, color='grey')
for s,e in best['pairs_crop']:ax.axvspan(s, e, alpha=0.1, color='red')
ax.plot(ages_crop, df_crop['d18O'], label='d18O')
ax.plot(ages_crop, sq_crop*np.max(abs(df_crop['d18O'])), label='GS signal')
ax.set_title(f"≤120 ka: Known vs Detected GS (F1={best['f1']:.2f})")
ax.set_xlabel("Age (yr)")
ax.legend()

# 7b. plot full record
fig2, ax2 = plt.subplots(figsize=(20,4))
for s,e in pairs_full:        ax2.axvspan(s, e, alpha=0.2, color='red')
ax2.plot(ages_full, new_df['d18O'], label='d18O')
ax2.plot(ages_full, sq_full*np.max(abs(new_df['d18O'])), label='GS signal')
ax2.set_title("Full Record: Detected GS")
ax2.set_xlabel("Age (yr)")
ax2.legend()

plt.tight_layout()
plt.show()
